## _PandaML_
- Perform detailed _Exploratory Data Analysis (EDA)_ on STT Data.
- Use `DASK` instead of `Pandas` to load CSVs at once.

In [1]:
import sys, os, glob, yaml

In [2]:
import math
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import trackml.dataset
import seaborn as sns

In [4]:
sys.path.append('src')

In [5]:
from src.utils_data import dask_events

In [6]:
import ROOT

Welcome to JupyROOT 6.26/08


In [7]:
%jsroot on

### _Dataset_

In [8]:
input_dir = './data_sets/ctd2022/train_100k/'

In [9]:
# Get List of all Files
all_files = os.listdir(input_dir)

# Extract File Prefixes (use e.g. xxx-hits.csv)
file_prefixes = sorted(os.path.join(input_dir, f.replace('-hits.csv', '')) for f in all_files if f.endswith('-hits.csv'))

In [10]:
print("Number of Files: ", len(file_prefixes))

Number of Files:  100000


In [11]:
# load an event
# hits, tubes, particles, truth = trackml.dataset.load_event(file_prefixes[0])

In [12]:
# tubes.head()

In [13]:
# particles.head()

In [14]:
# truth.head()

### _Read Event_

In [15]:
from src import SttCSVReader, Draw_Reader_Event

In [16]:
reader = SttCSVReader(input_dir, True, True)

In [17]:
data = reader(0)

In [18]:
data.event.columns

Index(['hit_id', 'x', 'y', 'z', 'volume_id', 'layer', 'module_id', 'event_id',
       'isochrone', 'skewed', 'sector_id', 'tx', 'ty', 'tz', 'tpx', 'tpy',
       'tpz', 'weight', 'particle_id', 'vx', 'vy', 'vz', 'px', 'py', 'pz', 'q',
       'nhits', 'pdgcode', 'start_time', 'pt', 'peta', 'r', 'phi', 'eta', 'r3',
       'absZ', 'tpt'],
      dtype='object')

In [19]:
# number of hits
data.event['nhits'].count()

229

In [20]:
# data.event.head()

In [21]:
# event = Compose_Event(file_prefixes[1], noise=False, skewed=True)

### Histograms

### _(1)-Number of Hits Histogram_

In [22]:
# Hit Counts
hc = ROOT.TH1F("hc", "Number of Hits;No. of Hits;Counts", 200, 100, 500)

In [ ]:
for i in range(len(file_prefixes)):
# for i in range(100):
    
    if i != 0 and i%1000 == 0:
        print("Processed Events:", i)
    
    data = reader(i)
    nhit = data.event["hit_id"].count()
    hc.Fill(nhit)

In [ ]:
can = ROOT.TCanvas("can","Histograms",600,500)

In [54]:
can.cd();hc.GetXaxis().SetTitleOffset(1.5);hc.Draw();can.Draw()#can.SaveAs("nhits.pdf");can.Close()

### _(2)-Particle Histograms_

In [19]:
# Hit Distributions
h1 = ROOT.TH1F("h1", "Number of Particles with pt > 50 MeV;No. of Particles;Counts", 100, 1, 10)
h2 = ROOT.TH1F("h2", "Number of Particles with pt > 50 MeV and nhits > 5;No. of Particles;Counts", 100, 1, 10)
h3 = ROOT.TH2F("h3", "Hit Distribution;x;y", 1000, -40, 40, 1000, -40, 40)

In [20]:
# Position Resolutions
hrx = ROOT.TH1F("hrx", "X Position Resolution;(x_{reco} - x_{true})/x_{true};Counts", 100, -10, 10)
hry = ROOT.TH1F("hry", "Y Position Resolution;(y_{reco} - y_{true})/y_{true};Counts", 100, -10, 10)
hrz = ROOT.TH1F("hrz", "Z Position Resolution;(z_{reco} - z_{true})/z_{true};Counts", 100, -10, 10)

In [21]:
for i in range(len(file_prefixes)):
    
    if i != 0 and i%1000 == 0:
        print("Processed Events:", i)
    
    data = reader(i)
    
    truth_particles = data.event
    truth_particles = truth_particles[truth_particles.pt > 0.05]       # particles with pt > 50 MeV
    h1.Fill(np.unique(truth_particles.particle_id).size)
    
    truth_particles = truth_particles[truth_particles.nhits > 5]       # particles with nhits > 4
    h2.Fill(np.unique(truth_particles.particle_id).size)
    
    xvals = data.event.x.values
    yvals = data.event.y.values
    
    r_x = ((data.event.x - data.event.tx)/data.event.tx).values
    r_y = ((data.event.y - data.event.ty)/data.event.ty).values
    r_z = ((data.event.z - data.event.tz)/data.event.tz).values
    
    for idx in range(data.event.shape[0]):
        h3.Fill(xvals[idx], yvals[idx])
        hrx.Fill(r_x[idx])
        hry.Fill(r_y[idx])
        hrz.Fill(r_z[idx])

Processed Events: 1000
Processed Events: 2000
Processed Events: 3000
Processed Events: 4000
Processed Events: 5000
Processed Events: 6000
Processed Events: 7000
Processed Events: 8000
Processed Events: 9000


In [23]:
c1 = ROOT.TCanvas("c1", "Histograms", 600,500)
c2 = ROOT.TCanvas("c2", "Histograms", 600,600)
c3 = ROOT.TCanvas("c3", "Histograms", 600,500)

In [24]:
c1.cd();h1.GetXaxis().SetTitleOffset(1.5);h1.Draw();c1.Draw();#c1.SaveAs("h1.pdf");c1.Close()

In [25]:
c1.cd();h2.GetXaxis().SetTitleOffset(1.5);h2.Draw();c1.Draw();#c1.SaveAs("h2.pdf");c1.Close()

In [26]:
c2.cd();h3.GetXaxis().SetTitleOffset(1.5);h3.GetYaxis().SetTitleOffset(1.5)h3.Draw("COLZ");c2.Draw();#c2.SaveAs("h3.pdf");c1.Close()

### _Position Resolutions_

In [27]:
c3.cd();hrx.GetXaxis().SetTitleOffset(1.5);hrx.Draw();c3.SetLogy(False);c3.Draw();# c3.SaveAs("hrx.pdf");c3.Close();

In [28]:
c3.cd();hry.GetXaxis().SetTitleOffset(1.5);hry.Draw();c3.SetLogy(False);c3.Draw();# c3.SaveAs("hry.pdf");c3.Close();

In [29]:
c3.cd();hrz.GetXaxis().SetTitleOffset(1.5);hrz.Draw();c3.SetLogy(False);c3.Draw();# c3.SaveAs("hrz.pdf");c3.Close();